In [1]:
import os
os.chdir("../../../")

import numpy as np
import scipy
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from src.tourism.utsmodel import *
from src.tourism.scaler import *
from src.tourism.mtsmodel import *
from src.tourism.ts_utils import *
from src.tourism.data import *
from src.tourism.cross_validate import TimeSeriesCrossValidator
from datetime import timedelta

import warnings
warnings.filterwarnings('ignore')

## SARIMAX

In [2]:
mod = SARIMAXPipeline(country="samoa",
                      training_ratio=1,
                      y_var="total",
                      exog_var=["covid", "stringency_index", "samoa_travel"],
                      transform_method="scaledlogit")
mod.read_and_merge()
mod.transform()
mod.stepwise_search()
mod_msres = mod.manual_search()

training size : 242, testing size : 0
Performing stepwise search to minimize aic
 ARIMA(0,1,0)(0,0,1)[12] intercept   : AIC=774.613, Time=0.15 sec
 ARIMA(0,1,0)(0,0,0)[12] intercept   : AIC=784.639, Time=0.02 sec
 ARIMA(1,1,0)(1,0,0)[12] intercept   : AIC=741.180, Time=0.14 sec
 ARIMA(0,1,1)(0,0,1)[12] intercept   : AIC=706.955, Time=0.35 sec
 ARIMA(0,1,0)(0,0,0)[12]             : AIC=782.839, Time=0.03 sec
 ARIMA(0,1,1)(0,0,0)[12] intercept   : AIC=715.476, Time=0.10 sec
 ARIMA(0,1,1)(1,0,1)[12] intercept   : AIC=700.864, Time=0.40 sec
 ARIMA(0,1,1)(1,0,0)[12] intercept   : AIC=702.062, Time=0.25 sec
 ARIMA(0,1,1)(2,0,1)[12] intercept   : AIC=699.908, Time=0.90 sec
 ARIMA(0,1,1)(2,0,0)[12] intercept   : AIC=700.217, Time=0.77 sec
 ARIMA(0,1,1)(2,0,2)[12] intercept   : AIC=702.503, Time=1.10 sec
 ARIMA(0,1,1)(1,0,2)[12] intercept   : AIC=700.140, Time=1.03 sec
 ARIMA(0,1,0)(2,0,1)[12] intercept   : AIC=755.510, Time=0.54 sec
 ARIMA(1,1,1)(2,0,1)[12] intercept   : AIC=689.637, Time=1.08

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1296/1296 [41:01<00:00,  1.90s/it]


In [3]:
for i in mod_msres[:10]:
    model_params = {
        "order": i[2][0],
        "seasonal_order": i[2][1]
    }
    tscv = TimeSeriesCrossValidator(method="SARIMAX",
                                    model_params=model_params,
                                    data=mod.y,
                                    exog_data=mod.exog,
                                    cv_method="Rolling",
                                    transformation="scaledlogit")
    
    initial_size = int(0.75*len(mod.y))
    cv_errors = tscv.cross_validate(hyper_params={"initial": initial_size})
    print(i[2], np.mean([error["RMSE"] for error in cv_errors]))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [01:25<00:00,  1.72s/it]


((4, 1, 5), (0, 1, 1, 12)) 3788.7693844108894


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [01:10<00:00,  1.40s/it]


((4, 1, 4), (0, 1, 1, 12)) 3812.402884212952


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [04:20<00:00,  5.21s/it]


...

((4, 1, 4), (1, 1, 2, 12)) 3831.0996343070237


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [02:51<00:00,  3.43s/it]


((3, 1, 2), (0, 1, 2, 12)) 3799.1191092948484


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [01:04<00:00,  1.29s/it]

((1, 1, 5), (0, 1, 1, 12)) 4011.7368472507555
